In [31]:
import tensorflow as tf
from tensorflow import keras
import IPython
import shutil

import kerastuner as kt

In [5]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4423680/4422102 [==============================] - ETA:  - 0s 0us/step


In [6]:
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

## Define the model

In [38]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=[28, 28]))
    
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    model.add(keras.layers.Dense(10))
    
    hp_lr = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=hp_lr),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[keras.metrics.sparse_categorical_accuracy]
    )
    
    return model

In [41]:
tuner = kt.Hyperband(
    hypermodel=model_builder,
    objective='val_sparse_categorical_accuracy',
    max_epochs=10,
    factor=3,
    directory='my_dir',
    project_name='intro_to_kt'
)

# The number of brackets is 1 + log_3(max_epochs), in our example is 3
# The cumulative epochs of all trials will be max_epochs * (math.log(max_epochs, factor) ** 2)

In [43]:
## Define a subclass of keras.callbacks.Callback to clear training output
class ClearTrainingOutput(keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait=True)
        
clear_training_output_cb = ClearTrainingOutput()

In [44]:
tuner.search(
    img_train,
    label_train,
    epochs=10,
    validation_data=(img_test, label_test),
    callbacks=[clear_training_output_cb]
)

INFO:tensorflow:Oracle triggered exit


In [22]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_hps.values

{'units': 96,
 'learning_rate': 0.01,
 'tuner/epochs': 4,
 'tuner/initial_epoch': 2,
 'tuner/bracket': 2,
 'tuner/round': 1,
 'tuner/trial_id': 'e9f0112caf67e646ad81932dd0e3d124'}

In [ ]:
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs=10, validation_data=(img_test, label_test))

In [46]:
shutil.rmtree('my_dir', ignore_errors=True)